In [2]:
import requests
import pandas as pd

# data.Rev.01
# https://docs.google.com/spreadsheets/d/1V_7tyAwn2S8KQ1B0OUPBGL_FgcgZgZtg-sqlp6IjcIA/edit?usp=sharing

file_id = "1V_7tyAwn2S8KQ1B0OUPBGL_FgcgZgZtg-sqlp6IjcIA" # Replace with your file ID
sheet_name = "Sheet1" # Replace with the name of your sheet

url = f"https://docs.google.com/spreadsheets/d/{file_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
res = requests.get(url)

with open("file.csv", "w") as f:
  f.write(res.text)

df = pd.read_csv("file.csv", skiprows=0)

In [3]:
# 날짜 형식 통일 및 datetime 객체로 변환
date_columns = ['대출신청월', '입주예정일']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

<ipython-input-3-37efa3d4bf98>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')
<ipython-input-3-37efa3d4bf98>:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [4]:
import pandas as pd
from datetime import datetime, timedelta

# 첫 번째 함수 이름 변경
def parse_date_standard(date_str):
    if pd.isna(date_str):
        return date_str
    try:
        for fmt in ("%Y년 %m월", "%Y-%m-%d", "%Y-%m", "%b-%d"):
            try:
                dt = datetime.strptime(date_str, fmt)
                return dt.strftime("%Y년 %m월")
            except ValueError:
                continue
        return date_str
    except Exception as e:
        return date_str

# 두 번째 함수는 기존대로 유지
def parse_date(date_str):
    for fmt in ("%Y년 %m월", "%Y-%m-%d", "%Y-%m"):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"date {date_str} is not in a recognized format")

# DataFrame의 모든 데이터에 대해 날짜 형식 통일
df = df.applymap(parse_date_standard)

# 원하는 열만 선택하여 새로운 DataFrame 생성
df = df.iloc[:, :16]

In [5]:
import pandas as pd
from datetime import datetime, timedelta

# 첫 번째 함수 이름 변경 및 datetime 객체로 반환
def parse_date_standard(date_str):
    if pd.isna(date_str):
        return date_str
    try:
        for fmt in ("%Y년 %m월", "%Y-%m-%d", "%Y-%m", "%b-%d"):
            try:
                dt = datetime.strptime(date_str, fmt)
                return dt
            except ValueError:
                continue
        return date_str
    except Exception as e:
        return date_str

# DataFrame의 모든 데이터에 대해 날짜 형식 통일 및 datetime 객체로 변환
df = df.applymap(parse_date_standard)

# 구매증빙서류(등기부등본_신규) 열 설정
def calculate_purchase_proof(row):
    if row['구매구분'] == '전세':
        return '미대상'
    elif row['구매구분'] == '구매':
        return (row['대출신청월'] + timedelta(days=3)).strftime('%Y년 %m월 %d일')  # 대출신청월에서 3개월 후 날짜 계산
    elif row['구매구분'] == '분양':
        return (row['대출신청월'] + timedelta(days=3)).strftime('%Y년 %m월 %d일')  # 대출신청월에서 3개월 후 날짜 계산
    else:
        return '미대상'

df['구매증빙서류(등기부등본_신규)'] = df.apply(calculate_purchase_proof, axis=1)

ValueError: NaTType does not support strftime

In [6]:
df.columns

Index(['이름', '사번', '직위', '유주택/무주택(서약서대상여부)', '구매구분', '대출신청월', '입주예정일',
       '유주택자 증빙서류(등기부등본_기존)', '확인_유주택자 증빙서류(등기부등본_기존)', '구매증빙서류(등기부등본_신규)',
       '확인_구매증빙서류(등기부등본_신규)', '구매증빙서류(주민등록등본_신규)', '확인_구매증빙서류(주민등록등본_신규)',
       '전세증빙서류(주민등록등본)', '확인_전세증빙서류(주민등록등본)', '서류제출완료여부'],
      dtype='object')

In [7]:
# 유주택자 증빙서류(등기부등본_기존) 열 설정
df['유주택자 증빙서류(등기부등본_기존)'] = df.apply(lambda row: 1 if row['유주택/무주택(서약서대상여부)'] == '대상' else '', axis=1)

# 구매증빙서류(등기부등본_신규) 열 설정
def calculate_purchase_proof(row):
    if row['구매구분'] == '전세':
        return '미대상'
    elif row['구매구분'] == '구매':
        return (row['대출신청월'] + timedelta(days=3)).strftime('%Y년 %m월 %d일')  # 대출신청월에서 3개월 후 날짜 계산
    elif row['구매구분'] == '분양':
        return (row['대출신청월'] + timedelta(days=3)).strftime('%Y년 %m월 %d일')  # 대출신청월에서 3개월 후 날짜 계산
    else:
        return '미대상'

df['구매증빙서류(등기부등본_신규)'] = df.apply(calculate_purchase_proof, axis=1)

# 구매증빙서류(주민등록등본_신규) 열 설정
df['구매증빙서류(주민등록등본_신규)'] = df.apply(lambda row: 1 if row['구매구분'] in ['구매', '분양'] else '', axis=1)

# 전세증빙서류(주민등록등본) 열 설정
df['전세증빙서류(주민등록등본)'] = df.apply(lambda row: 1 if row['구매구분'] == '전세' else '', axis=1)

# 서류제출완료여부 열 설정
df['서류제출완료여부'] = df.apply(lambda row: '완료' if all(row[['유주택자 증빙서류(등기부등본_기존)', '구매증빙서류(등기부등본_신규)', '구매증빙서류(주민등록등본_신규)', '전세증빙서류(주민등록등본)']]) else '', axis=1)

ValueError: NaTType does not support strftime

In [8]:
# 날짜 계산 함수
def calculate_date(base_date, offset_months, condition=True):
    if not condition or pd.isna(base_date):
        return "미대상"
    base_date_obj = parse_date(base_date)
    result_date = base_date_obj + timedelta(days=30*offset_months)
    return result_date.strftime("%Y년 %m월")

# 각 조건에 맞는 함수 적용
df['유주택자 증빙서류(등기부등본_기존)'] = df.apply(
    lambda row: calculate_date(row['대출신청월'], -1, row['유주택/무주택(서약서대상여부)'] == "대상"), axis=1)
df['구매증빙서류(등기부등본_신규)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] in ["구매", "분양"]), axis=1)
df['구매증빙서류(주민등록등본_신규)'] = df['구매증빙서류(등기부등본_신규)']
df['전세증빙서류(주민등록등본)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] == "전세"), axis=1)

# 서류제출완료여부 계산
def check_documents_status(row):
    documents = ['확인_유주택자 증빙서류(등기부등본_기존)', '확인_구매증빙서류(등기부등본_신규)', '확인_구매증빙서류(주민등록등본_신규)', '확인_전세증빙서류(주민등록등본)']
    counts = [1 if row[col] == 1 else 0 for col in documents]
    if sum(counts) == len(documents):
        return "완료"
    elif sum(counts) == 0:
        return "미진행중"
    else:
        return "진행중"

df['서류제출완료여부'] = df.apply(check_documents_status, axis=1)

TypeError: strptime() argument 1 must be str, not Timestamp

In [9]:
# 원하는 열만 선택하여 새로운 DataFrame 생성
df = df.iloc[:, :16]

In [10]:
df.columns

Index(['이름', '사번', '직위', '유주택/무주택(서약서대상여부)', '구매구분', '대출신청월', '입주예정일',
       '유주택자 증빙서류(등기부등본_기존)', '확인_유주택자 증빙서류(등기부등본_기존)', '구매증빙서류(등기부등본_신규)',
       '확인_구매증빙서류(등기부등본_신규)', '구매증빙서류(주민등록등본_신규)', '확인_구매증빙서류(주민등록등본_신규)',
       '전세증빙서류(주민등록등본)', '확인_전세증빙서류(주민등록등본)', '서류제출완료여부'],
      dtype='object')

In [11]:
# DataFrame을 CSV 파일로 저장
df.to_csv('output.csv', index=False, encoding='utf-8-sig')

In [12]:
# 날짜 형식에 따라 파싱을 돕는 함수
def parse_date(date_str):
    for fmt in ("%Y년 %m월", "%Y-%m-%d"):
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    raise ValueError(f"date {date_str} is not in a recognized format")

# 날짜 계산 함수
def calculate_date(base_date, offset_months, condition=True):
    if not condition or pd.isna(base_date):
        return "미대상"
    base_date_obj = parse_date(base_date)
    result_date = base_date_obj + timedelta(days=30*offset_months)
    return result_date.strftime("%Y년 %m월")

# 유주택자 증빙서류(등기부등본_기존), 구매증빙서류 등 계산
df['유주택자 증빙서류(등기부등본_기존)'] = df.apply(
    lambda row: calculate_date(row['대출신청월'], -1, row['유주택/무주택(서약서대상여부)'] == "대상"), axis=1)
df['구매증빙서류(등기부등본_신규)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] in ["구매", "분양"]), axis=1)
df['구매증빙서류(주민등록등본_신규)'] = df['구매증빙서류(등기부등본_신규)']
df['전세증빙서류(주민등록등본)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] == "전세"), axis=1)

# 서류제출완료여부 계산
def check_documents_status(row):
    documents = ['확인_유주택자', '확인_구매증빙', '확인_구매증빙_주민등록등본', '확인_전체증빙']
    counts = [1 if row[col] == 1 else 0 for col in documents]
    if sum(counts) == len(documents):
        return "완료"
    elif sum(counts) == 0:
        return "미진행중"
    else:
        return "진행중"

df['서류제출완료여부'] = df.apply(check_documents_status, axis=1)

# 결과 출력
print(df[['이름', '서류제출완료여부', '유주택자 증빙서류(등기부등본_기존)', '구매증빙서류(등기부등본_신규)', '전세증빙서류(주민등록등본)']])

TypeError: strptime() argument 1 must be str, not Timestamp

In [13]:
from datetime import datetime, timedelta

# 날짜 계산을 위한 함수 정의
def calculate_date(base_date, offset_months, condition=True):
    if not condition or pd.isna(base_date):
        return "미대상"
    date_format = "%Y년 %m월"
    base_date_obj = datetime.strptime(base_date, date_format)
    result_date = base_date_obj + timedelta(days=30*offset_months)
    return result_date.strftime(date_format)

# 유주택자 증빙서류(등기부등본_기존) 계산
df['유주택자 증빙서류(등기부등본_기존)'] = df.apply(
    lambda row: calculate_date(row['대출신청월'], -1, row['유주택/무주택(서약서대상여부)'] == "대상"), axis=1)

# 구매증빙서류(등기부등본_신규) 계산
df['구매증빙서류(등기부등본_신규)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] in ["구매", "분양"]), axis=1)

# 구매증빙서류(주민등록등본_신규) 계산
df['구매증빙서류(주민등록등본_신규)'] = df['구매증빙서류(등기부등본_신규)']

# 전세증빙서류(주민등록등본) 계산
df['전세증빙서류(주민등록등본)'] = df.apply(
    lambda row: calculate_date(row['입주예정일'], 3, row['구매구분'] == "전세"), axis=1)

# 서류제출완료여부 계산
def check_documents_status(row):
    documents = ['확인_유주택자', '확인_구매증빙', '확인_구매증빙_주민등록등본', '확인_전체증빙']
    counts = [1 if row[col] == 1 else 0 for col in documents]
    if sum(counts) == len(documents):
        return "완료"
    elif sum(counts) == 0:
        return "미진행중"
    else:
        return "진행중"

df['서류제출완료여부'] = df.apply(check_documents_status, axis=1)

# 결과 출력
print(df[['이름', '서류제출완료여부', '유주택자 증빙서류(등기부등본_기존)', '구매증빙서류(등기부등본_신규)', '전세증빙서류(주민등록등본)']])

TypeError: strptime() argument 1 must be str, not Timestamp